In [ ]:
pip install ratelimiter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import requests
from time import sleep
import datetime
from ratelimiter import RateLimiter
from google.colab import drive
import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#
airports = pd.read_csv("airports.csv", sep = ";")

In [ ]:
#creation des listes individuels 
np.random.seed(123)

ousmane , abdoul , abdel = np.array_split(np.random.permutation(airports["AirportCode"]), 3)

In [82]:
#collecte de flight_information
def get_flight_information(origins, dates, access_tokens):
    flights = []

    rate_limiter_seconds = RateLimiter(max_calls=4, period=1)  # 4 requêtes par seconde
    rate_limiter_hour = RateLimiter(max_calls=1000, period=3600)  # 1000 requêtes par heure
    api_key_index = 0
    requests_count = 0

    for date in dates:
        for origin in origins:
            for hour in range(24):
                for minute in ["00", "30"]:
                    with rate_limiter_seconds, rate_limiter_hour:
                        if requests_count >= 900:
                            print(f"Changement de clé API : utilisation de la clé {api_key_index + 1}") 
                            api_key_index += 1
                            requests_count = 0
                            if api_key_index >= len(access_tokens):
                                print("Toutes les clés API ont atteint la limite de requêtes.")
                                return pd.DataFrame.from_dict(flights).drop_duplicates().reset_index(drop=True)

                    time_str = f"{hour:02d}:{minute}"
                    datetime = f"{date}T{time_str}"
                    
                    authentification = {"Authorization": "Bearer " + access_tokens[api_key_index]}

                    try:
                        response = requests.get(
                            ("https://api.lufthansa.com/v1/operations/customerflightinformation/departures/{origin}/{datetime}"
                             "?limit=100&offset=0"
                            ).format(origin=origin, datetime=datetime),
                            headers=authentification).json()

                        for flight in response['FlightInformation']['Flights']['Flight']:
                            # Extraction des données du vol 
                            departure = flight.get('Departure', {})
                            arrival = flight.get('Arrival',{})
                            operatingCarrier = flight.get('OperatingCarrier')
                            equipment = flight.get('Equipment')
                            status = flight.get('Status')


                            row = {
                                'DpAirportCode': departure.get('AirportCode'),
                                'DpScheduledDate': departure.get('Scheduled', {}).get('Date'),
                                'DpScheduledTime': departure.get('Scheduled', {}).get('Time'),
                                'DpActualDate': departure.get('Actual', {}).get('Date'),
                                'DpActualTime': departure.get('Actual', {}).get('Time'),
                                'DpTerminalName': departure.get('Terminal', {}).get('Name'),
                                'DpTerminalGate': departure.get('Terminal', {}).get('Gate'),
                                'DpStatusCode': departure.get('Status', {}).get('Code'),
                                'DPStatusDescription': departure.get('Status', {}).get('Description'),
                                'ArrAirportCode':arrival.get('AirportCode', {}),
                                'ArrScheduledDate': arrival.get('Scheduled', {}).get('Date'),
                                'ArrpScheduledTime': arrival.get('Scheduled', {}).get('Time'),
                                'ArrActualDate':arrival.get('Actual', {}).get('Date'),
                                'ArrActualTime':arrival.get('Actual', {}).get('Time'),
                                'ArrTerminalName':arrival.get('Terminal', {}).get('Name'),
                                'ArrTerminalGate':arrival.get('Terminal', {}).get('Gate'),
                                'ArrStatusCode': arrival.get('Status', {}).get('Code'),
                                'ArrStatusDescription': arrival.get('Status', {}).get('Description'),                             
                                'AirlineID': operatingCarrier.get('AirlineID'),
                                'FlightNumber': operatingCarrier.get('FlightNumber'),
                                'AircraftCode': equipment.get('AircraftCode'),
                                'StatusCode' : status.get('Code'),
                                'StatusDescription' : status.get('Description')

                            }

                            flights.append(row)

                        requests_count += 1

                    except Exception as e:
                        print(f"no flight from origin {origin}, time {time_str}: {e}")

                    except requests.exceptions.Timeout:
                            print("Timeout error occurred. Waiting for 1 second before retrying.")
                            time.sleep(1)  # attendre une seconde
                            continue  # réessayer

    flight_information_departures = pd.DataFrame.from_dict(flights).reset_index(drop = True)
    flight_information_departures = flight_information_departures.drop_duplicates().reset_index(drop = True)
    flight_information_departures.insert(0, 'FlightId', flight_information_departures['AirlineID'] + flight_information_departures['FlightNumber'].astype(str))
    return flight_information_departures

In [83]:
origins = ousmane
dates = ["2023-05-28"]
access_tokens = ["cle1" , "cle2",
                 "cle3" , "cle4",
                 "cle5" , "cle6",
                 "cle7" , "cle8"]

flight_information_departures = get_flight_information(origins, dates, access_tokens)
flight_information_departures.to_csv("/content/drive/MyDrive/flight_information.csv", sep = ";", index=False)
display(flight_information_departures)

no flight from origin VNO, time 00:00: 'FlightInformation'
no flight from origin VNO, time 00:30: 'FlightInformation'
no flight from origin VNO, time 01:00: 'FlightInformation'
no flight from origin VNO, time 01:30: 'FlightInformation'
no flight from origin VNO, time 02:00: 'str' object has no attribute 'get'
no flight from origin VNO, time 02:30: 'str' object has no attribute 'get'
no flight from origin VNO, time 03:00: 'str' object has no attribute 'get'
no flight from origin VNO, time 03:30: 'str' object has no attribute 'get'
no flight from origin VNO, time 04:00: 'str' object has no attribute 'get'
no flight from origin VNO, time 04:30: 'str' object has no attribute 'get'
no flight from origin VNO, time 05:00: 'str' object has no attribute 'get'
no flight from origin VNO, time 05:30: 'str' object has no attribute 'get'
no flight from origin VNO, time 06:00: 'str' object has no attribute 'get'
no flight from origin VNO, time 06:30: 'FlightInformation'
no flight from origin VNO, tim

,FlightId,DpAirportCode,DpScheduledDate,DpScheduledTime,DpActualDate,DpActualTime,DpTerminalName,DpTerminalGate,DpStatusCode,DPStatusDescription,...,ArrActualTime,ArrTerminalName,ArrTerminalGate,ArrStatusCode,ArrStatusDescription,AirlineID,FlightNumber,AircraftCode,StatusCode,StatusDescription
0,LH887,VNO,2023-05-28,13:50,2023-05-28,14:38,None,A17,DP,Flight Departed,...,15:42,1,A38,LD,Flight Landed,LH,887,321,LD,Flight Landed
1,SN2372,VNO,2023-05-28,14:10,2023-05-28,14:06,None,None,DP,Flight Departed,...,15:33,None,None,LD,Flight Landed,SN,2372,319,LD,Flight Landed
2,LX1341,VNO,2023-05-28,14:40,2023-05-28,15:10,None,None,DP,Flight Departed,...,16:20,None,None,LD,Flight Landed,LX,1341,223,LD,Flight Landed
3,OS636,VNO,2023-05-28,14:55,2023-05-28,14:48,None,None,DP,Flight Departed,...,15:39,3,F21A,LD,Flight Landed,OS,636,320,LD,Flight Landed
4,2A3433,QFB,2023-05-28,05:52,2023-05-28,05:54,None,T01,DP,Flight Departed,...,06:58,None,None,LD,Flight Landed,2A,3433,TRN,LD,Flight Landed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,YY9453,QLJ,2023-05-28,15:09,2023-05-28,15:09,None,None,OT,Flight On Time,...,16:21,None,None,OT,Flight On Time,YY,9453,TRN,OT,Flight On Time
1025,YY9455,QLJ,2023-05-28,16:09,2023-05-28,16:09,None,None,OT,Flight On Time,...,17:21,None,None,OT,Flight On Time,YY,9455,TRN,OT,Flight On Time
1026,YY9457,QLJ,2023-05-28,18:09,2023-05-28,18:09,None,None,OT,Flight On Time,...,19:21,None,None,OT,Flight On Time,YY,9457,TRN,OT,Flight On Time
1027,YY9459,QLJ,2023-05-28,19:09,2023-05-28,19:09,None,None,OT,Flight On Time,...,20:21,None,None,OT,Flight On Time,YY,9459,TRN,OT,Flight On Time
